# Projeto: Engenharia de Dados e Garantia de Qualidade no Conjunto de Dados do Airbnb no Rio de Janeiro

#### Grupo5:
Mariana,
Gabriella,
Manu,
Fabio

In [ ]:
#instalar adaptador
# pip install psycopg2
#pip install great_expectations


SyntaxError: invalid syntax (287519625.py, line 3)

In [5]:
#importar bibliotecas
import pandas as pd
import psycopg2
import numpy as np
import great_expectations as ge

In [3]:
#mostrar todas as linhas e colunas dos dataframes
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Camada Bronze
A camada bronze salva no banco de dados do PostgreSQL contém os dados em seu formato original

In [4]:
#estabelecer conexão com a camada bronze
conn = psycopg2.connect(
    host = 'localhost', 
    database ="Bronze_Layer", 
    user ="postgres",  
    password = 131415  
)

In [ ]:
# # Função para consultar os dados e retornar como DataFrame
# def query(create_table = 'my_table', query= "SELECT * FROM calendar"):
#     # Cria o cursor para executar as consultas
#     cursor = conn.cursor()
#      # Execute a consulta e carregue o resultado em um DataFrame do pandas
#     df = pd.read_sql_query(query, conn)
#     return df

In [6]:
# criar cursor
cur_review = conn.cursor()
#selecionar conteúdo da tabela
cur_review.execute('SELECT * FROM reviews_table_bronze')
#selecionar todas as linhas
rows_review = cur_review.fetchall()
#transformar em dataframe
reviews = pd.DataFrame(rows_review, columns=['listing_id', 'id', 'date', 'reviewer_id', 'reviewer_name', 'comments'])

In [20]:
cur_listings = conn.cursor()
cur_listings.execute('SELECT * FROM listings_table_bronze')
rows_listings = cur_listings.fetchall()
listings = pd.DataFrame(rows_listings, columns=['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name', 'description', 'neighborhood_overview', 'picture_url', 'host_id', 
'host_url', 'host_name', 'host_since', 'host_location', 'host_about', 'host_response_time', 'host_response_rate', 'host_acceptance_rate', 
'host_is_superhost', 'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood', 'host_listings_count', 'host_total_listings_count', 
'host_verifications', 'host_has_profile_pic', 'host_identity_verified', 'neighbourhood', 'neighbourhood_cleansed', 
'neighbourhood_group_cleansed', 'latitude', 'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms', 'bathrooms_text', 
'bedrooms', 'beds', 'amenities', 'price', 'minimum_nights', 'maximum_nights', 'minimum_minimum_nights', 'maximum_minimum_nights', 
'minimum_maximum_nights', 'maximum_maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'calendar_updated', 
'has_availability', 'availability_30', 'availability_60', 'availability_90', 'availability_365', 'calendar_last_scraped', 
'number_of_reviews', 'number_of_reviews_ltm', 'number_of_reviews_l30d', 'first_review', 'last_review', 'review_scores_rating', 
'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 
'review_scores_value', 'license', 'instant_bookable', 'calculated_host_listings_count', 'calculated_host_listings_count_entire_homes', 
'calculated_host_listings_count_private_rooms', 'calculated_host_listings_count_shared_rooms', 'reviews_per_month'
])

In [15]:
cur_calendar = conn.cursor()
cur_calendar.execute('SELECT * FROM calendar_table_bronze')
rows_calendar = cur_calendar.fetchall()
calendar = pd.DataFrame(rows_calendar, columns=['listing_id', 'date', 'avaliable', 'price', 'adjusted_price', 'minimum_nights', 'maximum_nights'])

In [9]:
calendar = pd.read_csv('calendar.csv')
listings = pd.read_csv('listings.csv')
reviews = pd.read_csv('reviews.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'calendar.csv'

### Camada Silver
A camada Silver contém dados padronizados, limpos e métricas de qualidade implementadas

Data Clean - Camada Silver: <br>
- Identifique e lide com valores ausentes, duplicatas e outliers nos dados brutos da camada "bronze". <br>
- Padronize e limpe os nomes das colunas, convertendo-os em um formato consistente. <br>
- Realize uma limpeza textual em campos, como descrições de propriedades, removendo caracteres especiais e erros de digitação.

Data Quality - Camada Silver:
- Defina métricas de qualidade de dados, como integridade, precisão e consistência para os dados da camada "bronze".
- Implemente verificações para garantir que os dados da camada "silver" estejam em conformidade com essas métricas.
- Estabeleça um sistema de monitoramento contínuo da qualidade dos dados da camada "silver".

In [40]:
#converter reviews para os tipos corretos
reviews[['listing_id', 'id', 'reviewer_id']] = reviews[['listing_id', 'id', 'reviewer_id']].apply(pd.to_numeric)
reviews['date'] = pd.to_datetime(reviews['date'])

#Nulos e duplicados
reviews = reviews.dropna(how="all", axis=1).dropna(how="all")
reviews = reviews.drop_duplicates()

#remover outros caracteres especiais
remover = [']', '[', '"', '\n', '*', '\r', '<br/>']

for caractere in remover:
    for coluna in reviews.columns:
        if reviews[coluna].dtype == 'object':
            reviews[coluna] = reviews[coluna].str.replace(caractere, '', regex=False)

In [16]:
#Nulos e duplicados
calendar = calendar.dropna(how="all", axis=1).dropna(how="all")
calendar = calendar.drop_duplicates()

#converter calendar para os tipos corretos e remover caracteres especiais
calendar['price'] = calendar['price'].str.replace('$','', regex=False) #adjusted_price/price
calendar['price'] = calendar['price'].str.replace(',','', regex=False) #adjusted_price/price

# Converter para número
calendar[['listing_id', 'minimum_nights', 'maximum_nights','price']] = calendar[['listing_id', 'minimum_nights', 'maximum_nights','price']].apply(pd.to_numeric) #adjusted_price/price

# Converter 'date' para data
calendar['date'] = pd.to_datetime(calendar['date'])

# Converter "t" 'f' para True e False
calendar['avaliable'] = np.where(calendar['avaliable']=="t",True,False)

## Tratar valores nulos
for col in calendar.select_dtypes(include = "number").columns:
    calendar[col] = calendar[col].fillna(calendar[col].mean()).astype('int')


In [ ]:
# listings = pd.read_csv('listings.csv')
listings=listings[listings.host_location=='Rio de Janeiro, Brazil'] # Filtrar Rio de Janeiro

#Nulos e duplicados
listings = listings.dropna(how="all", axis=1).dropna(how="all")
listings = listings.drop_duplicates()

#converter listings para os tipos corretos e remover caracteres especiais
converter_numerico = [
    'id', 'host_id', 'host_listings_count', 'host_total_listings_count', 
    'accommodates', 'bathrooms', 'bedrooms', 'beds', 'minimum_nights', 
    'maximum_nights', 'minimum_minimum_nights', 'maximum_minimum_nights', 
    'minimum_maximum_nights', 'maximum_maximum_nights', 'minimum_nights_avg_ntm', 
    'maximum_nights_avg_ntm', 'availability_30', 'availability_60', 
    'availability_90', 'availability_365', 'number_of_reviews', 
    'number_of_reviews_ltm', 'number_of_reviews_l30d', 'review_scores_rating', 
    'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 
    'review_scores_communication', 'review_scores_location', 'review_scores_value', 
    'calculated_host_listings_count', 'calculated_host_listings_count_entire_homes', 
    'calculated_host_listings_count_private_rooms', 
    'calculated_host_listings_count_shared_rooms', 'reviews_per_month'
]

converter_data = [
    'last_scraped', 'host_since', 'first_review', 'last_review'
]

#converter os tipos conforme as listas acima
listings[converter_numerico] = listings[converter_numerico].apply(pd.to_numeric)
listings[converter_data] = listings[converter_data].apply(pd.to_datetime)

#corrigir a coluna de price que tinha caractere especial
listings['price'] = listings['price'].str.replace('$','', regex=False)
listings['price'] = listings['price'].str.replace(',','', regex=False).astype(float)

#existem valores N/A nas colunas host_response_rate e host_acceptance_rate; é prevciso alterar para NaN
listings['host_response_rate'] = listings['host_response_rate'].replace('N/A', np.NaN)
listings['host_acceptance_rate'] =listings['host_acceptance_rate'].replace('N/A', np.NaN)

#converter colunas com % para formato adequado
listings['host_response_rate'] = listings['host_response_rate'].str.replace('%', '', regex=False).astype(float) / 100
listings['host_acceptance_rate'] = listings['host_acceptance_rate'].str.replace('%', '', regex=False).astype(float) / 100

#algumas colunas de listings estão com vazio 'None', vou mudar para NaN
contem_vazios = ['description', 'neighborhood_overview', 'host_location', 'host_response_rate', 
'host_acceptance_rate', 'host_is_superhost', 'host_neighbourhood', 'neighbourhood',  'bathrooms', 'bathrooms_text', 'bedrooms', 
'beds', 'price', 'has_availability', 'first_review', 
'last_review', 'review_scores_rating', 'review_scores_accuracy', 
'review_scores_cleanliness', 'review_scores_checkin', 
'review_scores_communication', 'review_scores_location', 
'review_scores_value', 'reviews_per_month']

listings[contem_vazios] = listings[contem_vazios].fillna(np.nan)

#remover outros caracteres especiais
remover = [']', '[', '"', '\n', '*']

for caractere in remover:
    for coluna in listings.columns:
        if listings[coluna].dtype == 'object':
            listings[coluna] = listings[coluna].str.replace(caractere, '', regex=False)

# Converter True/False
for col in ['host_is_superhost','instant_bookable']:
    listings[col] = np.where(listings[col] =="t",True,False)

# Ajustando a coluna de verificações
expand = listings['host_verifications'].str.split(',', expand=True, n=3)
for col in expand.columns:
    expand[col] = expand[col].str.replace("'",'').str.replace("]",'').str.replace("[",'')

novos_nomes = {expand.columns[0]: 'host_verification_1',
               expand.columns[1]: 'host_verification_2',
               expand.columns[2]: 'host_verification_3'}


expand.rename(columns=novos_nomes, inplace=True)

listings = pd.concat([listings,expand],axis=1).drop("host_verifications",axis=1)    


Validação de Expectativas com Great Expectations - Camada Silver:
- Implemente validações adicionais usando Great Expectations nas camadas de dados da camada "silver".
- Monitore a qualidade dos dados da camada "silver" após cada transformação e ajuste os testes de acordo.

In [7]:
## Fazer o MErge em tudo para Great Expectations e depois subir na tabela ouro

merjao_bd_reviews = pd.merge(reviews, listings, left_on='listing_id' ,right_on='id',  how= 'left')
merjao_bd_reviews = pd.merge(merjao_bd_reviews, calendar, left_on='id_x', right_on='listing_id', how= 'left')

merjao_bd_reviews = merjao_bd_reviews.rename(columns = {'listing_id_x':'listing_id', 
                                                        'id_x':'id', 
                                                        'date_x':'date', 
                                                        'price_x':'price', 
                                                        'minimum_nights_x':'minimum_nights',
                                                        'maximum_nights_x':'maximum_nights'})

merjao_bd_reviews = merjao_bd_reviews.drop(columns=['listing_id_y', 'date_y', 'price_y', 'minimum_nights_y','maximum_nights_y','id_y'])


In [ ]:
#aqui entra o great expcectations ## EScolher colunas chave para metricas
great_expectations init

# Criar um contexto do Great Expectations
context = ge.data_context.DataContext("/path/to/great_expectations")  # Caminho para o seu projeto Great Expectations

# Supondo que você já tenha o DataFrame 'merjao_bd_reviews'
# Exemplo de DataFrame para fins ilustrativos:
# merjao_bd_reviews = pd.DataFrame(...)  # Seu DataFrame carregado

# Adicionar os dados como um "batch" para validação
batch_kwargs = {
    "dataset": merjao_bd_reviews,  # O seu DataFrame
    "batch_identifier": "batch_1",  # Identificador do batch, pode ser qualquer string única
}

# Criar o batch para validação
batch = context.get_batch(batch_kwargs)

# 1. Espera que a coluna 'host_location' tenha valores específicos
batch.expect_column_values_to_be_in_set("host_location", ['Rio de Janeiro, Brazil'])

# 2. Espera que a coluna 'id' tenha valores únicos
batch.expect_column_values_to_be_unique("id")

# 3. Espera que a coluna 'last_scraped' seja recente (últimos 12 meses)
# Definindo um intervalo de 12 meses atrás até o momento presente
twelve_months_ago = datetime.now() - timedelta(days=365)
batch.expect_column_values_to_be_between(
    "last_scraped",
    min_value=twelve_months_ago,  # Definindo o limite mínimo de 12 meses atrás
    max_value=datetime.now()  # Limite máximo (agora)
)

# Executar as validações e obter os resultados
results = batch.validate()

# Exibir os resultados da validação
print(results)


ModuleNotFoundError: No module named 'great_expectations'

In [ ]:
### Uma vez validados os resultados podemos subir a camada Gold para o Banco de dados:

## Subir dados no Banco
#selecionar conteúdo da tabela
cur_review.execute("""CREATE TABLE IF NOT EXISTS reviews_Gold (
    listing_id BIGINT NOT NULL,
    id BIGINT NOT NULL,
    date TIMESTAMP NOT NULL,
    reviewer_id BIGINT NOT NULL,
    reviewer_name TEXT,
    comments TEXT,
    listing_url TEXT,
    scrape_id FLOAT,
    last_scraped TIMESTAMP,
    source TEXT,
    name TEXT,
    description TEXT,
    neighborhood_overview TEXT,
    picture_url TEXT,
    host_id FLOAT,
    host_url TEXT,
    host_name TEXT,
    host_since TIMESTAMP,
    host_location TEXT,
    host_about TEXT,
    host_response_time TEXT,
    host_response_rate FLOAT,
    host_acceptance_rate FLOAT,
    host_is_superhost TEXT,
    host_thumbnail_url TEXT,
    host_picture_url TEXT,
    host_neighbourhood TEXT,
    host_listings_count FLOAT,
    host_total_listings_count FLOAT,
    host_has_profile_pic TEXT,
    host_identity_verified TEXT,
    neighbourhood TEXT,
    neighbourhood_cleansed TEXT,
    latitude FLOAT,
    longitude FLOAT,
    property_type TEXT,
    room_type TEXT,
    accommodates FLOAT,
    bathrooms FLOAT,
    bathrooms_text TEXT,
    bedrooms FLOAT,
    beds FLOAT,
    amenities TEXT,
    price FLOAT,
    minimum_nights FLOAT,
    maximum_nights FLOAT,
    minimum_minimum_nights FLOAT,
    maximum_minimum_nights FLOAT,
    minimum_maximum_nights FLOAT,
    maximum_maximum_nights FLOAT,
    minimum_nights_avg_ntm FLOAT,
    maximum_nights_avg_ntm FLOAT,
    has_availability TEXT,
    availability_30 FLOAT,
    availability_60 FLOAT,
    availability_90 FLOAT,
    availability_365 FLOAT,
    calendar_last_scraped TEXT,
    number_of_reviews FLOAT,
    number_of_reviews_ltm FLOAT,
    number_of_reviews_l30d FLOAT,
    first_review TIMESTAMP,
    last_review TIMESTAMP,
    review_scores_rating FLOAT,
    review_scores_accuracy FLOAT,
    review_scores_cleanliness FLOAT,
    review_scores_checkin FLOAT,
    review_scores_communication FLOAT,
    review_scores_location FLOAT,
    review_scores_value FLOAT,
    instant_bookable TEXT,
    calculated_host_listings_count FLOAT,
    calculated_host_listings_count_entire_homes FLOAT,
    calculated_host_listings_count_private_rooms FLOAT,
    calculated_host_listings_count_shared_rooms FLOAT,
    reviews_per_month FLOAT,
    host_verification_1 TEXT,
    host_verification_2 TEXT,
    host_verification_3 TEXT,
    available TEXT,
    PRIMARY KEY (id)
);
""")



In [ ]:
# Definindo a consulta de inserção em massa
insert_query = """
    INSERT INTO reviews_Gold (
        listing_id, id, date, reviewer_id, reviewer_name, comments, listing_url, scrape_id,
        last_scraped, source, name, description, neighborhood_overview, picture_url, host_id,
        host_url, host_name, host_since, host_location, host_about, host_response_time,
        host_response_rate, host_acceptance_rate, host_is_superhost, host_thumbnail_url,
        host_picture_url, host_neighbourhood, host_listings_count, host_total_listings_count,
        host_has_profile_pic, host_identity_verified, neighbourhood, neighbourhood_cleansed,
        latitude, longitude, property_type, room_type, accommodates, bathrooms, bathrooms_text,
        bedrooms, beds, amenities, price, minimum_nights, maximum_nights, minimum_minimum_nights,
        maximum_minimum_nights, minimum_maximum_nights, maximum_maximum_nights,
        minimum_nights_avg_ntm, maximum_nights_avg_ntm, has_availability, availability_30,
        availability_60, availability_90, availability_365, calendar_last_scraped,
        number_of_reviews, number_of_reviews_ltm, number_of_reviews_l30d, first_review,
        last_review, review_scores_rating, review_scores_accuracy, review_scores_cleanliness,
        review_scores_checkin, review_scores_communication, review_scores_location,
        review_scores_value, instant_bookable, calculated_host_listings_count,
        calculated_host_listings_count_entire_homes, calculated_host_listings_count_private_rooms,
        calculated_host_listings_count_shared_rooms, reviews_per_month, host_verification_1,
        host_verification_2, host_verification_3, available
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
        %s, %s, %s, %s, %s, %s, %s, %s);
"""

# Preparar os dados para inserção
for index, row in merjao_bd_reviews.iterrows():
    values = (
        row['listing_id'], row['id'], row['date'], row['reviewer_id'], row['reviewer_name'], row['comments'],
        row['listing_url'], row['scrape_id'], row['last_scraped'], row['source'], row['name'], row['description'],
        row['neighborhood_overview'], row['picture_url'], row['host_id'], row['host_url'], row['host_name'],
        row['host_since'], row['host_location'], row['host_about'], row['host_response_time'],
        row['host_response_rate'], row['host_acceptance_rate'], row['host_is_superhost'], row['host_thumbnail_url'],
        row['host_picture_url'], row['host_neighbourhood'], row['host_listings_count'],
        row['host_total_listings_count'], row['host_has_profile_pic'], row['host_identity_verified'],
        row['neighbourhood'], row['neighbourhood_cleansed'], row['latitude'], row['longitude'],
        row['property_type'], row['room_type'], row['accommodates'], row['bathrooms'], row['bathrooms_text'],
        row['bedrooms'], row['beds'], row['amenities'], row['price'], row['minimum_nights'],
        row['maximum_nights'], row['minimum_minimum_nights'], row['maximum_minimum_nights'],
        row['minimum_maximum_nights'], row['maximum_maximum_nights'], row['minimum_nights_avg_ntm'],
        row['maximum_nights_avg_ntm'], row['has_availability'], row['availability_30'], row['availability_60'],
        row['availability_90'], row['availability_365'], row['calendar_last_scraped'], row['number_of_reviews'],
        row['number_of_reviews_ltm'], row['number_of_reviews_l30d'], row['first_review'], row['last_review'],
        row['review_scores_rating'], row['review_scores_accuracy'], row['review_scores_cleanliness'],
        row['review_scores_checkin'], row['review_scores_communication'], row['review_scores_location'],
        row['review_scores_value'], row['instant_bookable'], row['calculated_host_listings_count'],
        row['calculated_host_listings_count_entire_homes'], row['calculated_host_listings_count_private_rooms'],
        row['calculated_host_listings_count_shared_rooms'], row['reviews_per_month'], row['host_verification_1'],
        row['host_verification_2'], row['host_verification_3'], row['available']
    )
    
    # Executar o comando de inserção
    cur_review.execute(insert_query, values)

# Confirmar a transação
connection.commit()


In [ ]:
cur_review.execute('SELECT * FROM reviews_table_bronze')

In [ ]:
# Fechar o cursor e a conexão
cur_review.close()
connection.close()


Transformação de Dados - Camada Gold:
- Crie a camada "gold" de dados, aplicando agregações especializadas, como médias de preços por propriedade, por período, e outras agregações especializadas.
- Armazene os dados da camada "gold" no mesmo banco de dados PostgreSQL, mantendo a estrutura de dados otimizada para consultas analíticas.